In [4]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'scripts'))
#print(sys.path)
import utils as u

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import countDistinct, col
import pyspark.sql.functions as F
import seaborn as sns

['/home/ruitwang/generic-buy-now-pay-later-project-group-3/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/ruitwang/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/home/ruitwang/.ipython', '/home/ruitwang/generic-buy-now-pay-later-project-group-3/scripts']


In [5]:
sp = (
    SparkSession.builder.appName("merchants")
    .config("spark.sql.session.timeZone", "+11")
    .config("spark.driver.memory", "10g")
    .config("spark.executor.memory", "10g")
    .config('spark.sql.parquet.cacheMetadata', 'True')
    .getOrCreate()
)

22/10/09 13:17:22 WARN Utils: Your hostname, Loky-PC resolves to a loopback address: 127.0.1.1; using 172.25.51.220 instead (on interface eth0)
22/10/09 13:17:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 13:17:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
merchants = u.read_processed(sp, "merchants")
merchants.toPandas()


22/10/09 13:21:00 ERROR Executor: Exception in task 6.0 in stage 5.0 (TID 17)
java.lang.IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: .part-00000-31a67f0d-224b-4d54-9b62-1fb7fdca92e8-c000.snappy.parquet:Zone.Identifier.crc
	at org.apache.hadoop.fs.Path.initialize(Path.java:263)
	at org.apache.hadoop.fs.Path.<init>(Path.java:221)
	at org.apache.hadoop.fs.Path.<init>(Path.java:129)
	at org.apache.hadoop.fs.ChecksumFileSystem.getChecksumFile(ChecksumFileSystem.java:107)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:163)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.parquet.hadoop.util.HadoopInputFile.newStream(HadoopInputFile.java:69)
	at org.apache.parquet.hadoop.ParquetFileReader.<init>(ParquetFileReader.java:773)
	at org.apache.parquet.hadoop.ParquetFileReader.open(ParquetFileReader.java:

Py4JJavaError: An error occurred while calling o73.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 5.0 failed 1 times, most recent failure: Lost task 6.0 in stage 5.0 (TID 17) (172.25.51.220 executor driver): java.lang.IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: .part-00000-31a67f0d-224b-4d54-9b62-1fb7fdca92e8-c000.snappy.parquet:Zone.Identifier.crc
	at org.apache.hadoop.fs.Path.initialize(Path.java:263)
	at org.apache.hadoop.fs.Path.<init>(Path.java:221)
	at org.apache.hadoop.fs.Path.<init>(Path.java:129)
	at org.apache.hadoop.fs.ChecksumFileSystem.getChecksumFile(ChecksumFileSystem.java:107)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:163)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.parquet.hadoop.util.HadoopInputFile.newStream